In [4]:
###将重采样样本进行合并
import pandas as pd
import glob
path = 'E:/BaiduYunDownload/sougou2/TT/'

whole_data = glob.glob(path+'*.csv')
train_resam = pd.DataFrame()
for filename in whole_data:
    fil = pd.read_csv(filename,encoding = 'gb18030')
    train_resam = train_resam.append(fil,ignore_index=True)
train_resam = train_resam.rename(columns={'Unnamed: 0':'ID'})
train_resam = train_resam[['ID','age','Gender','Education','Query']]

In [5]:
###数据处理
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from scipy.sparse import csr_matrix, hstack ,vstack
np.random.seed(1)
import jieba 
from collections import Counter
import jieba.posseg as pseg
import jieba.analyse
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import svm
from multiprocessing.dummy import Pool as ThreadPool
from sklearn import preprocessing
from bs4 import BeautifulSoup
from sklearn.linear_model import RidgeClassifier,Lasso
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,precision_score

def mean_len(L):
    num = len(L)-1 if len(L)>1 else 1
    sum_len = sum(L)-max(L)
    return round(float(sum_len)/num,1)

train = pd.read_table('E:/BaiduYunDownload/sougou2/user_tag_query.10W.TRAIN',encoding = 'gb18030',header = None,sep = '\n',names = ['col'])
test = pd.read_table('E:/BaiduYunDownload/sougou2/user_tag_query.10W.TEST',encoding = 'gb18030',header = None,sep = '\n',names = ['col'])
train['ID'] = train['col'].apply(lambda x:x.split('\t')[0])
train['age'] = train['col'].apply(lambda x:x.split('\t')[1])
train['Gender'] = train['col'].apply(lambda x:x.split('\t')[2])
train['Education'] =  train['col'].apply(lambda x:x.split('\t')[3])
train['Query'] = train['col'].apply(lambda x:','.join(x.split('\t')[4::]))
train = pd.concat([train,train_resam])
train = shuffle(train)
####统计特征

train['mean'] = train['Query'].apply(lambda x:mean_len([len(i) for i in x.split(',')]))
train['max'] = train['Query'].apply(lambda x:max([len(i) for i in x.split(',')]))
train['std'] = train['Query'].apply(lambda x:round(np.array(([len(i) for i in x.split(',')])).std(),1))
train['sum'] = train['Query'].apply(lambda x:np.array(([len(i) for i in x.split(',')])).sum())

train['age'] = train['age'].astype(int)
train['Gender'] = train['Gender'].astype(int)
train['Education'] = train['Education'].astype(int)

test['ID'] = test['col'].apply(lambda x:x.split('\t')[0])
test['Query'] = test['col'].apply(lambda x:','.join(x.split('\t')[1::]))
####统计特征

test['mean'] = test['Query'].apply(lambda x:mean_len([len(i) for i in x.split(',')]))
test['max'] = test['Query'].apply(lambda x:max([len(i) for i in x.split(',')]))
test['std'] = test['Query'].apply(lambda x:round(np.array(([len(i) for i in x.split(',')])).std(),1))
test['sum'] = test['Query'].apply(lambda x:np.array(([len(i) for i in x.split(',')])).sum())

train_data = train.drop('col',axis = 1)
test_data = test.drop('col',axis = 1)
test_data.insert(1,'age',0)
test_data.insert(2,'Gender',0)
test_data.insert(3,'Education',0)
test_data['TT'] = 'test'
train_data['TT'] = 'train'

train = train_data.drop(list(train_data[(train_data['age']==0)|(train_data['Gender']==0)|(train_data['Education']==0)].index),axis = 0)
###根据预测结果及样本分布，删除age为6，edu=1,2的所有数据，但删除后并无改善
data = pd.concat([train,test_data])
data = data.reset_index().drop('index',axis = 1)
data = data.drop(data[data['age']==6].index,axis = 0)
data = data.drop(data[data['Education']==1].index,axis = 0)
data = data.drop(data[data['Education']==2].index,axis = 0)
#data = data.drop(data[(data['age']==1)&(data['Education']==2)].index,axis = 0)

In [6]:
data.shape

(200685, 10)

In [7]:
path = 'E:/BaiduYunDownload/sougou/Freq/'
#jieba.load_userdict("E:/BaiduYunDownload/sougou/Freq/1.txt")
jieba.analyse.set_stop_words(path+'stop_wd.txt')
def fenci(data):
    list_fenci = []
    for content in data.index:
        feature_seg = jieba.cut(data['Query'][content])
        seg_join = ','.join(feature_seg)
        list_fenci.append(seg_join)
    return list_fenci
pool0 = ThreadPool(8)
l_data = [data]
fenci = pool0.map(fenci,l_data)
data['jieba_fenci'] = fenci[0]

Building prefix dict from the default dictionary ...
Loading model from cache c:\users\admini~1\appdata\local\temp\jieba.cache
Loading model cost 0.285 seconds.
Prefix dict has been built succesfully.


In [8]:
vectorizer1 = CountVectorizer(ngram_range= (1,2),min_df=18,max_df = 0.75,)
vectorizer2 = CountVectorizer(ngram_range= (1,2),min_df=25,max_df = 0.6)
vectorizer3 = CountVectorizer(ngram_range= (1,2),min_df=18,max_df = 0.85,)#(ngram_range= (1,2),min_df=10,max_df = 0.9,)
X_vec1 = vectorizer1.fit_transform(data['jieba_fenci'].values)
X_vec2 = vectorizer2.fit_transform(data['jieba_fenci'].values)
X_vec3 = vectorizer3.fit_transform(data['jieba_fenci'].values)
X1 = TfidfTransformer(use_idf = False,smooth_idf = True,sublinear_tf=0.6).fit_transform(X_vec1)
X2 = TfidfTransformer(use_idf = True,smooth_idf = True,sublinear_tf=0.8).fit_transform(X_vec2)
X3 = TfidfTransformer(use_idf = True,smooth_idf = True,sublinear_tf=0.6).fit_transform(X_vec3)

In [9]:
minmax = preprocessing.MinMaxScaler()
mean1 = minmax.fit_transform(data.loc[:,'mean']) 
max1 = minmax.fit_transform(data.loc[:,'max']) 
std1 = minmax.fit_transform(data.loc[:,'std'])
sum1 = minmax.fit_transform(data.loc[:,'sum'])

cs = csr_matrix(np.hstack(( np.atleast_2d(mean1).T,np.atleast_2d(std1).T,np.atleast_2d(max1).T,np.atleast_2d(sum1).T
                          )))
X1 = hstack((X1, cs),format='csr')
X2 = hstack((X2, cs),format='csr')
X3 = hstack((X3, cs),format='csr')

D:\Anaconda2\lib\site-packages\scikit_learn-0.18.1-py2.7-win-amd64.egg\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
D:\Anaconda2\lib\site-packages\scikit_learn-0.18.1-py2.7-win-amd64.egg\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
D:\Anaconda2\lib\site-packages\scikit_learn-0.18.1-py2.7-win-amd64.egg\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  war

In [12]:
X3

<200685x459449 sparse matrix of type '<type 'numpy.float64'>'
	with 64075660 stored elements in Compressed Sparse Row format>

In [13]:
y_a = np.asarray(data['age'].values)
y_g = np.asarray(data['Gender'].values)
y_e = np.asarray(data['Education'].values)
from sklearn.feature_selection import SelectKBest, chi2
X_new1 = SelectKBest(chi2, k=X1.shape[1]/7).fit_transform(X1, y_a)
X_new2 = SelectKBest(chi2, k=X2.shape[1]/6).fit_transform(X2, y_g)
X_new3 = SelectKBest(chi2, k=X3.shape[1]/7).fit_transform(X3, y_e)

In [14]:
y1 = np.asarray(data[data['TT']=='train']['age'].values)
y2 = np.asarray(data[data['TT']=='train']['Gender'].values)
y3 = np.asarray(data[data['TT']=='train']['Education'].values)

data_train1, data_test1, target_train1, target_test1 = cross_validation.train_test_split(X_new1[0:(data.shape[0]-100000)], y1)
data_train2, data_test2, target_train2, target_test2 = cross_validation.train_test_split(X_new2[0:(data.shape[0]-100000)], y2)
data_train3, data_test3, target_train3, target_test3 = cross_validation.train_test_split(X_new3[0:(data.shape[0]-100000)], y3)
sgd1 = SGDClassifier(penalty = 'elasticnet',n_jobs=-1,random_state=1)
sgd1.fit(data_train1,target_train1)
predict1 = sgd1.predict(data_test1)
lr2 = LogisticRegression(n_jobs = -1,random_state=1)
lr2.fit(data_train2,target_train2)
predict2 = lr2.predict(data_test2)
sgd3 = SGDClassifier(penalty = 'elasticnet',n_jobs=-1,random_state=1)
sgd3.fit(data_train3,target_train3)
predict3 = sgd3.predict(data_test3)

precision_score1 = precision_score(target_test1, predict1,average = 'micro')
precision_score2 = precision_score(target_test2, predict2,average = 'micro')
precision_score3 = precision_score(target_test3, predict3,average = 'micro')
print "\nModel Report"
print "precision_score : %.4g" % ((precision_score1+precision_score2+precision_score3)/3)


Model Report
precision_score : 0.6901


In [15]:
###svm可以使复赛结果达到0.703+
import time
from multiprocessing.dummy import Pool as ThreadPool
from sklearn.svm import SVC

start = time.time()
def predict(X):
    svm =SVC(kernel="linear")
    svm.fit(X[1][0:(data.shape[0]-100000)],X[0])
    prediction = svm.predict(X[1][(data.shape[0]-100000):data.shape[0]])
    return prediction
x1 = [y1,X_new1]
x2 = [y2,X_new2]
x3 = [y3,X_new3]
L1 = [x1,x2,x3]
pool = ThreadPool(8)
result = pool.map(predict,L1)
pool.close()
pool.join()
print("Time: {:.2f} seconds".format(time.time() - start))

Time: 28765.55 seconds


In [16]:
submission = pd.DataFrame()
submission['ID'] = data[data['TT']=='test']['ID']
submission['age'] = result[0]
submission['Gender'] = result[1]
submission['Education'] = result[2]
submission.to_csv('E:/BaiduYunDownload/sougou2/12.10svm.csv',sep = ' ',index = False,header = None,encoding ='GBK')

In [ ]:
###stacking 非常耗时
import random

def run(data):
    X = data[0]
    Y = data[1]
    
    dev_cutoff = 105802
    X_dev = X[:dev_cutoff]
    Y_dev = Y[:dev_cutoff]
    X_test = X[dev_cutoff:]
    Y_test = Y[dev_cutoff:]

    n_folds = 5
    
    clfs = [
        SVC(kernel="linear"),
        svm.LinearSVC(),
        SGDClassifier(loss = 'hinge'),
        SGDClassifier(penalty = 'elasticnet'),
        MultinomialNB(),
        BernoulliNB(),
        RidgeClassifier(),
        
    ]
    
    skf = list(StratifiedKFold(Y_dev, n_folds))
    
    blend_train = np.zeros((X_dev.shape[0], len(clfs))) 
    blend_test = np.zeros((X_test.shape[0], len(clfs))) 
    
    print 'X_test.shape = %s' % (str(X_test.shape))
    print 'blend_train.shape = %s' % (str(blend_train.shape))
    print 'blend_test.shape = %s' % (str(blend_test.shape))
    
    for j, clf in enumerate(clfs):
        print 'Training classifier [%s]' % (j)
        blend_test_j = np.zeros((X_test.shape[0], len(skf))) 
        for i, (train_index, cv_index) in enumerate(skf):
            print 'Fold [%s]' % (i)
            
            X_train = X_dev[train_index]
            Y_train = Y_dev[train_index]
            X_cv = X_dev[cv_index]
            Y_cv = Y_dev[cv_index]
            
            clf.fit(X_train, Y_train)
            
            blend_train[cv_index, j] = clf.predict(X_cv)
            blend_test_j[:, i] = clf.predict(X_test)
        
        blend_test[:, j] = blend_test_j.mean(1)
    
    print 'Y_dev.shape = %s' % (Y_dev.shape)
    
    bclf = LogisticRegression()
    bclf.fit(blend_train, Y_dev)
    
    Y_test_predict = bclf.predict(blend_test)
    
    return Y_test_predict

l1 = [X_new1,y1]
l2 = [X_new2,y2]
l3 = [X_new3,y3]
list1 = [l1,l2,l3]
pool = ThreadPool(8)
stacking_result = pool.map(run,list1)    

In [ ]:
submission1 = pd.DataFrame()
submission1['ID'] = test['ID']
submission1['age'] = stacking_result[0]
submission1['Gender'] = stacking_result[1]
submission1['Education'] = stacking_result[2]
submission1.to_csv('E:/BaiduYunDownload/sougou2/12.02stacking.csv',sep = ' ',index = False,header = None,encoding ='GBK')